In [1]:
import qiskit
from qiskit import QuantumRegister, ClassicalRegister, assemble
from qiskit.circuit import Gate
from qiskit.circuit.library import XGate, HGate
from qiskit.providers.aer.pulse import PulseSystemModel
from src import qoc_instruction_schedule_map
from src.QOC_optimizer import QOC_optimizer

from qiskit.providers.aer.pulse.hamiltonian_model import HamiltonianModel
from qutip import identity, sigmax, sigmaz, sigmay
import qutip.control.pulseoptim as cpo
import qutip.logging_utils as logging
import numpy as np

from qutip.qip.device import Processor
from qutip.states import basis
from qutip.operators import identity

In [2]:
import qutip.control.pulseoptim as cpo


In [3]:
from qiskit import IBMQ, pulse, QuantumCircuit
from qiskit.pulse import Play
# import qiskit

IBMQ.load_account()

provider = IBMQ.get_provider(hub='ibm-q', group='open', project='main')
backend = provider.get_backend('ibmq_armonk')
armonk_backend = backend
# original_backend = provider.get_backend('ibmq_armonk')

In [4]:
armonk_backend.name()

'ibmq_armonk'

In [5]:

# set the qubit frequency
# freq_est = 4.97e9
freq_est = 4974450442.586346
wq0 = 2 * np.pi * freq_est
delta = wq0 / 16
delta_freq = delta / 2 / np.pi

In [6]:

from src.qoc_instruction_schedule_map import QOCInstructionScheduleMap
hamiltonian = {'H_d': delta * (1-sigmaz())/2, 'H_c': [31919806.545849085 * sigmax()]}
# grape_calibrator = QOC_optimizer(cpo, hamiltonian)

In [7]:
from src.QutipOptimizer import QutipOptimizer
from src.helper import qutip_ham_converter
# unitary_goal = sigmax()
qutip_ham_converter(armonk_backend)
# grape_calibrator = QOC_optimizer(cpo, hamiltonian)
grape_calibrator = QutipOptimizer(hamiltonian)
new_inst_map = QOCInstructionScheduleMap(grape_calibrator)
# new_inst_map.get(XGate(), [0])

NotImplementedError: 

In [ ]:
# x = XGate()
# x.to_matrix()

new_x = HGate()
new_x.name = HGate()

In [ ]:
q = QuantumRegister(1)
c = ClassicalRegister(1)
circ = qiskit.QuantumCircuit(q, c)
# circ.x(q)
# circ.x(q)
# circ.append(XGate(),[0])
circ.append(new_x,[0])


circ.measure([0], [0])

In [ ]:
# circ.data
#
# new_inst_map._map
# inst_map._

In [ ]:
#so every time we take a new gate, then we have to store in optimizer

In [ ]:
#TODO: figure out how to fit in this part (the defaults
new_inst_map._map = backend.defaults().instruction_schedule_map._map
new_inst_map._qubit_instructions = backend.defaults().instruction_schedule_map._qubit_instructions

In [ ]:
# set the drive strength
getattr(armonk_backend.configuration(), 'hamiltonian')['vars']['omegad0'] = 31919806.54584908

# set the qubit frequency
armonk_backend.configuration().qubit_freq_est=[freq_est-delta_freq]
getattr(armonk_backend.configuration(), 'hamiltonian')['vars']['wq0'] = 2*np.pi*freq_est

In [ ]:
pulse_aer_backend = qiskit.providers.aer.PulseSimulator(
    configuration=backend.configuration())

def_inst = backend.defaults().instruction_schedule_map
new_sched = qiskit.schedule(circ, inst_map=new_inst_map, backend=pulse_aer_backend, meas_map=[[0]])
# def_sched = qiskit.schedule(circ, inst_map=def_inst, backend=pulse_aer_backend, meas_map=[[0]])
# my_transp_circ = qiskit.transpile(circ, backend=pulse_aer_backend, basis_gates=armonk_backend.configuration().basis_gates + ['opt_gate'])

In [ ]:
armonk_model = PulseSystemModel.from_backend(armonk_backend)

# small_transp_circ = qiskit.transpile(circ, backend=armonk_backend, basis_gates=armonk_backend.configuration().basis_gates + ['opt_gate'])
small_gate_sched = qiskit.schedule(circ, inst_map=new_inst_map, backend=pulse_aer_backend, meas_map=[[0]])

# program = assemble(small_gate_sched, backend=armonk_backend, meas_level=2, meas_return='single',
#                    shots=4096, qubit_lo_freq=[freq_est-delta_freq])
program = assemble(new_sched, backend=armonk_backend, meas_level=2, meas_return='single',
                   shots=4096, qubit_lo_freq=[freq_est-delta_freq])
#
job = qiskit.execute(small_gate_sched, pulse_aer_backend, shots=4096, system_model=armonk_model)
# prog_job = qiskit.execute(program, pulse_aer_backend, shots=4096, system_model=armonk_model)
prog_job = pulse_aer_backend.run(program,armonk_model)

In [ ]:
# circ.draw()
prog_job.result().get_counts()

In [ ]:
job.result().get_counts()

In [ ]:
job = qiskit.execute(new_sched, pulse_aer_backend, shots=4096, system_model=PulseSystemModel.from_backend(armonk_backend))
# job = qiskit.execute(def_sched, pulse_aer_backend, shots=4096, system_model=PulseSystemModel.from_backend(armonk_backend))
job.result().get_counts()

In [ ]:
# my_gate = Gate('x', 0, )
# from qiskit.extensions import UnitaryGate
#
    # @deprecate_arguments({'q': 'qubit'})
    # def y(self, qubit, *, q=None):  # pylint: disable=unused-argument
    #     """Apply :class:`~qiskit.circuit.library.YGate`."""
    #     from .library.standard_gates.y import YGate
    #     return self.append(YGate(), [qubit], [])
# my_gate = UnitaryGate(sigmax())